---
title: Importing data using dhis2-python-client
short_title: data import with python client
---

This section walks you through importing data values into DHIS2 using the **`dhis2-python-client`** library.

**Below are crucial steps to follow:**
- Get credentials
- Connect to DHIS2 and verify access
- Build a valid `dataValueSets` payload (JSON or CSV)
- Send your payload

## 1) Configure your environment and connect to DHIS2

In [19]:
import pandas as pd
from dhis2_client import DHIS2Client
from dhis2_client.settings import ClientSettings

# Client configuration
cfg = ClientSettings(
  base_url="http://localhost:8080",
  username="admin",
  password="district")

client = DHIS2Client(settings=cfg)
info = client.get_system_info()

# Check if everything is working.
# You should see your current DHIS2 version info.
print("▶ Current DHIS2 version:", info["version"])


▶ Current DHIS2 version: 2.42.2-SNAPSHOT


## 2) Prepare metadata

There are two paths to prepare your payload. The first and probably easier path is to use existing metadata, i.e. organisation units, data elements, data sets, period and related category and attribute option combos if available. The second one is to create all of these from scratch and `dhis2-python-client` allows you to do this easily. For exmaple below is a simple step to create a organisation unit:

In [10]:
ou = {"name": "My organisation unit", "shortName": "My org unit", "openingDate": "2020-01-01"}
org_unit_response = client.create_org_unit(ou)
print(f"▶ Organizational unit create status: {org_unit_response['status']} and UID: {org_unit_response['response']['uid']}")


▶ Organizational unit create status: OK and UID: at4fZqDz0lW


Since we haven't assigned a parent to the above org unit, it will be created at the root level. For subsquent org units, we could assign `parent': {'id': 'at4fZqDz0lW'}` and they would be placed underneath `My organisation unit`. This way we could create the entire org unit tree. Once org units are created, we can proceed to creating data elements:

In [ ]:
data_element = {
    "name": "My Sample data element",
    "shortName": "Sample data element",
    "valueType": "NUMBER",
    "aggregationType": "SUM",
    "domainType": "AGGREGATE"
}
data_element_response = client.create_data_element(data_element)
print(f"▶ Data element create status: {data_element_response['status']} and UID: {data_element_response['response']['uid']}")

▶ Data element create status: OK and UID: i9x1eWzeZzS


Next in the metadata creation process is `data sets`. Data sets are crucial in defining how often we will be collecting value for a data element, i.e. they help us define `periods`. Not only that, data sets also help us connect a data element with an org unit. Below is how to do this:

In [ ]:
data_set = {
    "name": "My sample data set", 
    "shortName": "Sample data set",
    "periodType": "Monthly",
    "dataSetElements": [
        {
            "dataElement": {"id": data_element_response['response']['uid']}
        }
    ],
    "organisationUnits": [
        {
            "id": org_unit_response['response']['uid']
        }
    ]
}

data_set_response = client.create_data_set(data_set)
print(f"▶ Data set create status: {data_set_response['status']} and UID: {data_set_response['response']['uid']}")

▶ Data set create status: OK and UID: lFKYnmcAse0


Once data set is created, next step is to grant dat awrite access. The following can give current user data write access

In [ ]:
sharing = client.grant_self_data_write_on_dataset(data_set_response['response']['uid'])

print("✔ Granted myself data write on the dataset", sharing)

At this point you should have your metadata ready to accept data. Next step is to construct data payload and do a POST.

## 3) Construct payload

Data values can be sent to DHIS2 one-by-one or in batch. The one-by-one approach is very handy espeically when users are entering data manually. For batch import DHIS2 provides `dataValueSets` endpint. Below is a sample payload demonstrating batch data import.

In [ ]:
payload = {
    "dataValues": [
        {
            "dataElement": data_element_response['response']['uid'],
            "orgUnit": org_unit_response['response']['uid'],
            "period": "202501",
            "value": 2501
        },
        {
            "dataElement": data_element_response['response']['uid'],
            "orgUnit": org_unit_response['response']['uid'],
            "period": "202502",
            "value": 2502
        }
    ]
}

## 4) Send your payload

Once we are done preparing our payload we can proceed to using send it to DHIS2. `dhis2-python-client` provides both direct access to raw DHIS2 API like `client.post(/api/dataValueSets, ...)` or convenient method `client.post_data_value_set(...)`. Since we saw the convenient methods in the above metadata creation steps let's use the raw DHIS2 API this time.

In [ ]:
res = client.post("/api/dataValueSets", json=payload)
print("▶ Data value set post status: ", res)

▶ Data value set post status:  {'httpStatus': 'OK', 'httpStatusCode': 200, 'status': 'OK', 'message': 'Import was successful.', 'response': {'status': 'SUCCESS', 'importOptions': {'idSchemes': {}, 'dryRun': False, 'async': False, 'importStrategy': 'CREATE_AND_UPDATE', 'mergeMode': 'REPLACE', 'reportMode': 'FULL', 'skipExistingCheck': False, 'sharing': False, 'skipNotifications': False, 'skipAudit': False, 'datasetAllowsPeriods': False, 'strictPeriods': False, 'strictDataElements': False, 'strictCategoryOptionCombos': False, 'strictAttributeOptionCombos': False, 'strictOrganisationUnits': False, 'strictDataSetApproval': False, 'strictDataSetLocking': False, 'strictDataSetInputPeriods': False, 'requireCategoryOptionCombo': False, 'requireAttributeOptionCombo': False, 'skipPatternValidation': False, 'ignoreEmptyCollection': False, 'force': False, 'firstRowIsHeader': True, 'skipLastUpdated': False, 'mergeDataValues': False, 'skipCache': False}, 'description': 'Import process completed succ


## 5) Troubleshooting

- **Unauthorized**: Check credentials and user permissions.  
- **Not found**: Verify data element, org unit, and combos (if you have used non-default ones) exist.  
- **Conflicts**: Ensure dataset assignments and period are correct.  
- **Locked periods**: Unlock dataset period if needed.  
- **Value types**: Match the data element value type.  
